Import libraries

In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import glob
from collections import defaultdict
import matplotlib.pyplot as plt
from hoi.metrics import InfoTopo, TC, DTC, Sinfo, Oinfo, RedundancyphiID, SynergyphiID
from hoi.core import entropy_hist

Define functions

In [2]:
# --- Core Functions ---
def compute_information_metric(metric_class, data):
    """Compute information metric on data using a given metric class."""
    data_array = np.array(data, dtype=float)
    model = metric_class(data_array)
    result = model.fit(minsize=2, maxsize=4)
    return result, model

def divide_list_by_order(values, model):
    """Divide values into groups by interaction order."""
    orders = model.order
    grouped = defaultdict(list)
    for order, val in zip(orders, values):
        grouped[order].append(val.item())
    return {k: sorted(v) for k, v in grouped.items()}

def analyse_information_metric(metric_class, data):
    """Analyze data using a specified metric and return grouped results."""
    values, model = compute_information_metric(metric_class, data)
    return divide_list_by_order(values, model)

def sliding_window_analysis(metric_class, timeseries, window_size, step_size):
    """Apply a metric using a sliding window over a time series."""
    results = []
    for start in range(0, len(timeseries) - window_size, step_size):
        window_data = timeseries[start:start + window_size]
        result = analyse_information_metric(metric_class, window_data)
        results.append(dict(result))
    return results

def average_metric_data(simulations):
    """Prepare averaged metric data for heatmap visualization."""
    key_indices = set(
        (k, idx) for sim in simulations for step in sim for k, v in step.items() for idx in range(len(v))
    )
    sorted_keys = sorted(key_indices)

    data = []
    for t in range(len(simulations[0])):
        avg_step = {
            (k, idx): np.mean([sim[t].get(k, [0.0]*(idx+1))[idx] for sim in simulations])
            for k, idx in sorted_keys
        }
        data.append([avg_step[(k, idx)] for k, idx in sorted_keys])

    label_pos = {k: max(i for i, (key, _) in enumerate(sorted_keys) if key == k) for k, _ in sorted_keys}
    labels = list(label_pos.keys())

    return np.array(data), labels, label_pos

def compute_simulation_averages(metric_sims):
    """Average metric values across simulations and time."""
    all_sim_averages = [
        [np.mean([val for values in step.values() for val in values]) for step in sim]
        for sim in metric_sims
    ]
    combined_avg = np.mean(all_sim_averages, axis=0)
    return (*all_sim_averages, combined_avg)

# --- Data Loading ---
def load_csv_files_from_directory(directory):
    return glob.glob(os.path.join(directory, '*.csv'))

def read_and_trim_csv(file_path, num_rows=100):
    return pd.read_csv(file_path).iloc[:num_rows]

# --- Visualization Settings ---
def configure_plot_fonts(base_size=20):
    plt.rcParams.update({
        'font.size': base_size + 2,
        'axes.titlesize': base_size + 4,
        'axes.labelsize': base_size + 2,
        'xtick.labelsize': base_size,
        'ytick.labelsize': base_size,
        'legend.fontsize': base_size
    })

def plot_metric_results(metrics_data, averages_data, metrics_classes):
    """Visualize line plots and heatmaps for each metric."""
    num_metrics = len(metrics_data)
    fig, axs = plt.subplots(2, num_metrics, figsize=(6 * num_metrics, 10), sharey='row')
    axs = np.array(axs).reshape(2, num_metrics)

    timesteps = range(1, len(metrics_data[0][0]) + 1)

    for i, (top_ax, bottom_ax, sim_avg, (data, labels, label_pos), metric_cls) in enumerate(zip(
        axs[0], axs[1], averages_data, metrics_data, metrics_classes
    )):
        *individual_avgs, avg_all = sim_avg

        # Line plot
        for avg in individual_avgs:
            top_ax.plot(timesteps, avg, color='gray', linestyle='--', marker='o', alpha=0.7)
        top_ax.plot(timesteps, avg_all, color='red', marker='o', linewidth=2)
        top_ax.set_title(metric_cls.__name__)
        top_ax.set_xticklabels([])
        if i == 0:
            top_ax.set_ylabel("Average Value")
        else:
            top_ax.set_yticklabels([])
        top_ax.grid(True)

        # Heatmap
        cax = bottom_ax.imshow(data.T, aspect='auto', cmap='viridis', origin='lower')
        bottom_ax.set_xlabel("Time")
        bottom_ax.set_xticks(range(data.shape[0]))
        bottom_ax.set_xticklabels(range(data.shape[0]))
        if i == 0:
            bottom_ax.set_ylabel("Order")
            bottom_ax.set_yticks(list(label_pos.values()))
            bottom_ax.set_yticklabels(labels)

    plt.tight_layout(h_pad=0.1)
    plt.show()

# --- Main Execution ---
def main(data_dir, metrics_classes=[InfoTopo, TC, Oinfo], timeseries_percentage=1, window_percentage=0.5, step_percentage=0.1):
    files = load_csv_files_from_directory(data_dir)

    metric_results = []
    for metric in metrics_classes:
        all_sims = []
        for file_path in files:
            df = read_and_trim_csv(file_path)
            df_length = int(timeseries_percentage * len(df))
            df = df[:df_length]
            window = int(window_percentage * len(df))
            step = int(step_percentage * len(df))
            sim_result = sliding_window_analysis(metric, df, window, step)
            all_sims.append(sim_result)
        metric_results.append(all_sims)

    heatmap_data = [average_metric_data(m) for m in metric_results]
    averages_data = [compute_simulation_averages(m) for m in metric_results]

    heatmap_path = os.path.join(data_dir, "heatmap_data.pkl")
    averages_path = os.path.join(data_dir, "averages_data.pkl")

    with open(heatmap_path, "wb") as f:
        pickle.dump(heatmap_data, f)

    with open(averages_path, "wb") as f:
        pickle.dump(averages_data, f)

Check the length of the timeseries in the desired dataset

In [ ]:
directory = r'Data\Artificial_data\Redundant_timeseries'
csv_files = glob.glob(os.path.join(directory, '*.csv'))
df = pd.read_csv(csv_files[0])
len(df)

Compute and store the desired metrics

In [4]:
data_directory = r'Data\Artificial_data\Synergistic_timeseries'
metrics = [InfoTopo, TC, DTC, Sinfo, Oinfo, RedundancyphiID, SynergyphiID]
timeseries_percentage = 1
window_percentage = 0.5
step_percentage = 0.05

main(data_directory, metrics, timeseries_percentage, window_percentage, step_percentage)

Compute entropy with gc
    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/4 [00:00<?,       ?it/s]

Get list of multiplets


  0%|          | 0/375 [00:00<?, ?it/s]

Compute entropy with gc
    Copnorm and demean the data
Get list of multiplets


...

  0%|          |  0/3 [00:00<?,       ?it/s]

    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/3 [00:00<?,       ?it/s]

    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/3 [00:00<?,       ?it/s]